In [ ]:
"""Загрузка библиотек и данных из репозитория"""

import os
import cv2
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Dense
from keras import Model
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import load_img, img_to_array
from keras.utils import to_categorical
from keras.metrics import Recall

!git clone https://github.com/Laz-eg/Sick-chicken-detection.git

Cloning into 'Sick-chicken-detection'...
remote: Enumerating objects: 3186, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 3186 (delta 0), reused 19 (delta 0), pack-reused 3167
Receiving objects: 100% (3186/3186), 181.82 MiB | 20.95 MiB/s, done.
Updating files: 100% (3158/3158), done.


In [ ]:
"""Подгрузка аннотаций и их предобработка"""

def load_annotations(annotations_path):
    annot_frame = pd.read_csv(annotations_path, header=None)
    annot_frame.columns = ['filename', 'x1', 'y1', 'x2', 'y2', 'class']
    annot_frame['x'] = annot_frame[['x1', 'x2']].min(axis=1)
    annot_frame['y'] = annot_frame[['y1', 'y2']].min(axis=1)
    annot_frame['w'] = annot_frame[['x1', 'x2']].max(axis=1) - annot_frame['x']
    annot_frame['h'] = annot_frame[['y1', 'y2']].max(axis=1) - annot_frame['y']
    return annot_frame

train_path = '/content/Sick-chicken-detection/data/chicken detector/train'
valid_path = '/content/Sick-chicken-detection/data/chicken detector/valid'
test_path = '/content/Sick-chicken-detection/data/chicken detector/test'

train_annotations_path = os.path.join(train_path, '_annotations.csv')
valid_annotations_path = os.path.join(valid_path, '_annotations.csv')
test_annotations_path = os.path.join(test_path, '_annotations.csv')

train_annotations = load_annotations(train_annotations_path)
valid_annotations = load_annotations(valid_annotations_path)
test_annotations = load_annotations(test_annotations_path)

display(train_annotations.head())
display(valid_annotations.head())
display(test_annotations.head())

,filename,x1,y1,x2,y2,class,x,y,w,h
0,-107-_jpg.rf.881a88ce270bfe3b16493b9af0116ed5.jpg,0,1,318,233,chicken,0,1,318,232
1,280_jpg.rf.884e92e448af6cdb87a6255730bf7bb5.jpg,8,59,384,366,chicken,8,59,376,307
2,-53-_jpg.rf.888111b1e368172c9805a25a29e21fcc.jpg,189,140,317,270,chicken,189,140,128,130
3,-53-_jpg.rf.888111b1e368172c9805a25a29e21fcc.jpg,121,81,296,176,chicken,121,81,175,95
4,-53-_jpg.rf.888111b1e368172c9805a25a29e21fcc.jpg,67,159,205,275,chicken,67,159,138,116


,filename,x1,y1,x2,y2,class,x,y,w,h
0,-6-_mp4-27_jpg.rf.00d3869ee79bddce67604af59d45...,199,141,327,317,chicken,199,141,128,176
1,-6-_mp4-27_jpg.rf.00d3869ee79bddce67604af59d45...,68,104,200,340,chicken,68,104,132,236
2,-271-_jpg.rf.0157a23b45b8e90bc664cdf99264d0a6.jpg,234,59,374,260,chicken,234,59,140,201
3,video7_58_jpg.rf.03861c003a94721ed8f3e8c06a75e...,222,215,355,403,chicken,222,215,133,188
4,video7_58_jpg.rf.03861c003a94721ed8f3e8c06a75e...,0,155,83,416,chicken,0,155,83,261


,filename,x1,y1,x2,y2,class,x,y,w,h
0,-155-_jpg.rf.2151e1bcf3415b389bb1629befe8dfdb.jpg,164,58,366,388,chicken,164,58,202,330
1,chicks16_jpg.rf.13b12c14b6481a68c150f24186deea...,133,170,272,374,chicken,133,170,139,204
2,chicks16_jpg.rf.13b12c14b6481a68c150f24186deea...,91,0,187,132,chicken,91,0,96,132
3,broiler-chicken-days-old-isolated-white-132140...,32,47,321,364,chicken,32,47,289,317
4,OIP-26-_jpeg.rf.15c9dcf2395343138b5af4736395c4...,127,41,305,416,chicken,127,41,178,375


In [ ]:
"""Загрузка изображений в программу"""
# Функция для загрузки изображений из локальной папки
def load_images_from_folder(folder_path, limit=20):
    images = []
    filenames = []
    for idx, filename in enumerate(os.listdir(folder_path)):
        if idx >= limit:
            break
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder_path, filename)
            #print(f"Загрузка изображения: {img_path}")
            try:
                img = load_img(img_path)
                if img is not None:
                    images.append(img_to_array(img))
                    filenames.append(filename)
            except Exception as e:
                print(f"Ошибка загрузки изображения {img_path}: {e}")
    return images, filenames

# Подгрузка изображений
train_size = 200
test_size = 30
valid_size = 30

train_images, train_filenames = load_images_from_folder(train_path, limit=train_size)
test_images, test_filenames = load_images_from_folder(test_path, limit=test_size)
valid_images, valid_filenames = load_images_from_folder(valid_path, limit=valid_size)

print(f"Загружено {len(train_images)} изображений для тренировки")
print(f"Загружено {len(test_images)} изображений для тестирования")
print(f"Загружено {len(valid_images)} изображений для валидации")

Загружено 200 изображений для тренировки
Загружено 30 изображений для тестирования
Загружено 30 изображений для валидации


In [ ]:
"""Функции создания и подготовки данных для обучения модели"""

# Функция для расчета IoU (Intersection over Union)
def compute_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    xi1 = max(x1, x2)
    yi1 = max(y1, y2)
    xi2 = min(x1 + w1, x2 + w2)
    yi2 = min(y1 + h1, y2 + h2)
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = w1 * h1
    box2_area = w2 * h2
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area

# Функция для автосегментации изображений
def selective_search(image):
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(image)
    ss.switchToSelectiveSearchFast()
    rects = ss.process()
    return rects

# Функция создания выборки
def create_dataset(images, filenames, annotations, iou_threshold_positive=0.85, iou_threshold_negative=0.1, N=3):
    X = []
    Y = []
    for img, filename in zip(images, filenames):
        img_array = np.array(img, dtype=np.uint8)
        rects = selective_search(img_array)
        pos_samples = 0
        neg_samples = 0
        temp_X_pos = []
        temp_Y_pos = []
        temp_X_neg = []
        temp_Y_neg = []
        for (x, y, w, h) in rects:
            roi = img_array[y:y+h, x:x+w]
            roi_resized = cv2.resize(roi, (224, 224))
            max_iou = 0
            found_positive = False
            for idx, row in annotations[annotations['filename'] == filename].iterrows():
                iou = compute_iou((x, y, w, h), (row['x'], row['y'], row['w'], row['h']))
                if iou > max_iou:
                    max_iou = iou
                if iou >= iou_threshold_positive:
                    found_positive = True
            if found_positive:
                temp_X_pos.append(roi_resized)
                temp_Y_pos.append(np.array([1, 0]))  # Курица
                pos_samples += 1
            elif max_iou <= iou_threshold_negative:
                temp_X_neg.append(roi_resized)
                temp_Y_neg.append(np.array([0, 1]))  # Не курица
                neg_samples += 1

        if neg_samples > 2 * pos_samples:
            neg_samples = 2 * pos_samples
            temp_X_neg = temp_X_neg[:neg_samples]
            temp_Y_neg = temp_Y_neg[:neg_samples]

        X.extend(temp_X_pos)
        Y.extend(temp_Y_pos)
        X.extend(temp_X_neg)
        Y.extend(temp_Y_neg)

    print(f"Создано {len(X)} образцов.")
    return np.array(X), np.array(Y)

In [ ]:
"""Формирование выборок"""
train_X, train_y = create_dataset(train_images, train_filenames, train_annotations[train_annotations['filename'].isin(train_filenames)])
valid_X, valid_y = create_dataset(valid_images, valid_filenames, valid_annotations[valid_annotations['filename'].isin(valid_filenames)])
test_X, test_y = create_dataset(test_images, test_filenames, test_annotations[test_annotations['filename'].isin(test_filenames)])

# Преобразование меток в категориальный формат
# train_y = to_categorical(train_y, num_classes=2)
# valid_y = to_categorical(valid_y, num_classes=2)
# test_y = to_categorical(valid_y, num_classes=2)

print(f"Создано {train_X.shape[0]} тренировочных образцов")
print(f"Создано {valid_X.shape[0]} валидационных образцов")
print(f"Создано {test_X.shape[0]} валидационных образцов")

# Проверка данных перед созданием генераторов
if train_X.shape[0] == 0:
  raise ValueError("Нет данных для обучения. Проверьте загрузку изображений и аннотаций.")

if valid_X.shape[0] == 0:
  raise ValueError("Нет данных для валидации. Проверьте загрузку изображений и аннотаций.")

if test_X.shape[0] == 0:
  raise ValueError("Нет данных для валидации. Проверьте загрузку изображений и аннотаций.")

Создано 2406 образцов.
Создано 618 образцов.
Создано 342 образцов.
Создано 2406 тренировочных образцов
Создано 618 валидационных образцов
Создано 342 валидационных образцов


In [ ]:
"""Базовая модель"""
basic_model = VGG16(weights='imagenet', include_top=True)

553467096/553467096 [==============================] - 5s 0us/step


In [ ]:
"""Параметры для оптимизации"""
# Количество изображений: train - 200, test - 30, valid - 30
lr = 0.00001 # best = 0.00001 -- скорость обучения оптимизатора

train_batch_size = 64 # best = 64 -- размер входных пачек изображений при обучении
train_steps = 26  # best = 26 -- количество шагов обучения

valid_batch_size = 32 # best = 32 -- размер входных пачек изображений при валидации
valid_steps = 8 # best = 8 -- количество шагов валидации

num_epochs = 4 # best = 4 -- число эпох обучения

if train_batch_size*train_steps > train_X.shape[0]:
  print('Выбрано слишком много шагов для тренировочной выборки')
  train_steps = train_X.shape[0]//train_batch_size
  print(f'Количество шагов уменьшено до {train_steps}')

if valid_batch_size*valid_steps > valid_X.shape[0]:
  print('Выбрано слишком много шагов для валидационной выборки')
  valid_steps = valid_X.shape[0]//valid_batch_size
  print(f'Количество шагов уменьшено до {valid_steps}')

In [ ]:
"""Архитектура детектора"""
for layers in (basic_model.layers)[:15]:
    layers.trainable = False
X = basic_model.layers[-2].output
predictions = Dense(2, activation="softmax")(X)
detector = Model(inputs=basic_model.input, outputs=predictions)
opt = Adam(learning_rate=lr)
detector.compile(loss=keras.losses.categorical_crossentropy, optimizer=opt, metrics=[Recall()])  # Используем Recall как метрику
detector.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
"""Построение генераторов изображений для обучения нейросети"""

print(train_X.shape)
print(valid_X.shape)

trdata = ImageDataGenerator()
traindata = trdata.flow(x = train_X, y = train_y, batch_size=train_batch_size)
vdata = ImageDataGenerator()
validdata = vdata.flow(x = valid_X, y = valid_y, batch_size=valid_batch_size)

(2406, 224, 224, 3)
(618, 224, 224, 3)


In [ ]:
"""Обучение модели"""

detector.fit(
    traindata,
    steps_per_epoch=train_steps,
    epochs=num_epochs,
    validation_data=validdata,
    validation_steps=valid_steps)

Epoch 1/4
26/26 [==============================] - 42s 1s/step - loss: 0.1599 - recall: 0.9493 - val_loss: 0.1760 - val_recall: 0.9531
Epoch 2/4
26/26 [==============================] - 10s 363ms/step - loss: 0.0249 - recall: 0.9927 - val_loss: 0.1017 - val_recall: 0.9727
Epoch 3/4
26/26 [==============================] - 10s 376ms/step - loss: 0.0135 - recall: 0.9969 - val_loss: 0.0946 - val_recall: 0.9727
Epoch 4/4
26/26 [==============================] - 10s 394ms/step - loss: 0.0017 - recall: 1.0000 - val_loss: 0.0887 - val_recall: 0.9766


In [ ]:
"""Тестирование обученной модели"""
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

y_predict = detector.predict(test_X)
y_predict_rounded = np.round(y_predict, decimals=0).astype(int) # округление для получения ответа, какой это класс

length = y_predict.shape[0]*y_predict.shape[1]  # "выпрямление" массива в одномерный для подсчета метрик
y_test_1d = test_y.reshape(length, )[0::2]
y_predict_1d = y_predict_rounded.reshape(length, )[0::2]

accuracy = accuracy_score(y_test_1d, y_predict_1d)
precision = precision_score(y_test_1d, y_predict_1d)
recall = recall_score(y_test_1d, y_predict_1d)
matrix = confusion_matrix(y_test_1d, y_predict_1d)

print(f"accuracy = {accuracy}")
print(f"precision = {precision}")
print(f"recall = {recall}")
print('')
print('confusion matrix:')
display(matrix)

11/11 [==============================] - 7s 655ms/step
accuracy = 0.97953216374269
precision = 0.9819819819819819
recall = 0.956140350877193

confusion matrix:


array([[226,   2],
       [  5, 109]])

In [ ]:
"""Сохранение успешной модели"""
# Точность - 97.5%
# detector.save('detector_97.h5')

'Сохранение успешной модели'

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')